# Image Segmentation Task 1
#### Welcome to the first task of Image Segmentation. Image segmentation is the process of partitioning the image into a set of pixels representing an object. In this task, you will be introduced to the problem of image segmentation and programming pipeline involved in image segmentation.

For the purpose of this task we will be using PASCAL VOC datset. The dataset contains a total of 2913 images with segmentation annotations. Code in the cell below will download the code and extract the dataset.

In [1]:
# !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
# !tar -xvf VOCtrainval_11-May-2012.tar

In [2]:
# !pip install scipy==1.1.0

### 1.1 Loading the dataset

In [3]:
# They took this loader from here: 
# https://github.com/meetshah1995/pytorch-semseg/tree/801fb200547caa5b0d91b8dde56b837da029f746/ptsemseg/loader

import os
from os.path import join as pjoin
import collections
import json
import torch
import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob
import torch.nn as nn

from PIL import Image
from tqdm import tqdm
from torch.utils import data
from torchvision import transforms


class pascalVOCDataset(data.Dataset):
    """Data loader for the Pascal VOC semantic segmentation dataset.

    Annotations from both the original VOC data (which consist of RGB images
    in which colours map to specific classes) and the SBD (Berkely) dataset
    (where annotations are stored as .mat files) are converted into a common
    `label_mask` format.  Under this format, each mask is an (M,N) array of
    integer values from 0 to 21, where 0 represents the background class.

    The label masks are stored in a new folder, called `pre_encoded`, which
    is added as a subdirectory of the `SegmentationClass` folder in the
    original Pascal VOC data layout.

    A total of five data splits are provided for working with the VOC data:
        train: The original VOC 2012 training data - 1464 images
        val: The original VOC 2012 validation data - 1449 images
        trainval: The combination of `train` and `val` - 2913 images
        train_aug: The unique images present in both the train split and
                   training images from SBD: - 8829 images (the unique members
                   of the result of combining lists of length 1464 and 8498)
        train_aug_val: The original VOC 2012 validation data minus the images
                   present in `train_aug` (This is done with the same logic as
                   the validation set used in FCN PAMI paper, but with VOC 2012
                   rather than VOC 2011) - 904 images
    """

    def __init__(
        self,
        root,
        sbd_path=None,
        split="train_aug",
        is_transform=False,
        img_size=512,
        augmentations=None,
        img_norm=True,
        test_mode=False,
    ):
        self.root = root
        self.sbd_path = sbd_path
        self.split = split
        self.is_transform = is_transform
        self.augmentations = augmentations
        self.img_norm = img_norm
        self.test_mode = test_mode
        self.n_classes = 21
        self.mean = np.array([104.00699, 116.66877, 122.67892])
        self.files = collections.defaultdict(list)
        self.img_size = img_size if isinstance(img_size, tuple) else (img_size, img_size)

        if not self.test_mode:
            for split in ["train", "val", "trainval"]:
                path = pjoin(self.root, "ImageSets/Segmentation", split + ".txt")
                file_list = tuple(open(path, "r"))
                file_list = [id_.rstrip() for id_ in file_list]
                self.files[split] = file_list
            self.setup_annotations()

        self.tf = transforms.Compose(
            [
                # add more trasnformations as you see fit
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        im_name = self.files[self.split][index]
        im_path = pjoin(self.root, "JPEGImages", im_name + ".jpg")
        lbl_path = pjoin(self.root, "SegmentationClass/pre_encoded", im_name + ".png")
        im = Image.open(im_path)
        lbl = Image.open(lbl_path)
        if self.augmentations is not None:
            im, lbl = self.augmentations(im, lbl)
        if self.is_transform:
            im, lbl = self.transform(im, lbl)
        return im, torch.clamp(lbl, max=20)

    def transform(self, img, lbl):
        if self.img_size == ("same", "same"):
            pass
        else:
            img = img.resize((self.img_size[0], self.img_size[1]))  # uint8 with RGB mode
            lbl = lbl.resize((self.img_size[0], self.img_size[1]))
        img = self.tf(img)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl[lbl == 255] = 0
        return img, lbl

    def get_pascal_labels(self):
        """Load the mapping that associates pascal classes with label colors

        Returns:
            np.ndarray with dimensions (21, 3)
        """
        return np.asarray(
            [
                [0, 0, 0],
                [128, 0, 0],
                [0, 128, 0],
                [128, 128, 0],
                [0, 0, 128],
                [128, 0, 128],
                [0, 128, 128],
                [128, 128, 128],
                [64, 0, 0],
                [192, 0, 0],
                [64, 128, 0],
                [192, 128, 0],
                [64, 0, 128],
                [192, 0, 128],
                [64, 128, 128],
                [192, 128, 128],
                [0, 64, 0],
                [128, 64, 0],
                [0, 192, 0],
                [128, 192, 0],
                [0, 64, 128],
            ]
        )

    def encode_segmap(self, mask):
        """Encode segmentation label images as pascal classes

        Args:
            mask (np.ndarray): raw segmentation label image of dimension
              (M, N, 3), in which the Pascal classes are encoded as colours.

        Returns:
            (np.ndarray): class map with dimensions (M,N), where the value at
            a given location is the integer denoting the class index.
        """
        mask = mask.astype(int)
        label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
        for ii, label in enumerate(self.get_pascal_labels()):
            label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
        label_mask = label_mask.astype(int)
        # print(np.unique(label_mask))
        return label_mask

    def decode_segmap(self, label_mask, plot=False):
        """Decode segmentation class labels into a color image

        Args:
            label_mask (np.ndarray): an (M,N) array of integer values denoting
              the class label at each spatial location.
            plot (bool, optional): whether to show the resulting color image
              in a figure.

        Returns:
            (np.ndarray, optional): the resulting decoded color image.
        """
        label_colours = self.get_pascal_labels()
        r = label_mask.copy()
        g = label_mask.copy()
        b = label_mask.copy()
        for ll in range(0, self.n_classes):
            r[label_mask == ll] = label_colours[ll, 0]
            g[label_mask == ll] = label_colours[ll, 1]
            b[label_mask == ll] = label_colours[ll, 2]
        rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
        rgb[:, :, 0] = r / 255.0
        rgb[:, :, 1] = g / 255.0
        rgb[:, :, 2] = b / 255.0
        if plot:
            plt.imshow(rgb)
            plt.show()
        else:
            return rgb

    def setup_annotations(self):
        """Sets up Berkley annotations by adding image indices to the
        `train_aug` split and pre-encode all segmentation labels into the
        common label_mask format (if this has not already been done). This
        function also defines the `train_aug` and `train_aug_val` data splits
        according to the description in the class docstring
        """
        sbd_path = self.sbd_path
        target_path = pjoin(self.root, "SegmentationClass/pre_encoded")
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        train_aug = self.files["train"]

        # keep unique elements (stable)
        train_aug = [train_aug[i] for i in sorted(np.unique(train_aug, return_index=True)[1])]
        self.files["train_aug"] = train_aug
        set_diff = set(self.files["val"]) - set(train_aug)  # remove overlap
        self.files["train_aug_val"] = list(set_diff)

        pre_encoded = glob.glob(pjoin(target_path, "*.png"))
        expected = np.unique(self.files["train_aug"] + self.files["val"]).size

        if len(pre_encoded) != expected:
            print("Pre-encoding segmentation masks...")

            for ii in tqdm(self.files["trainval"]):
                fname = ii + ".png"
                lbl_path = pjoin(self.root, "SegmentationClass", fname)
                lbl = self.encode_segmap(m.imread(lbl_path))
                lbl = m.toimage(lbl, high=lbl.max(), low=lbl.min())
                m.imsave(pjoin(target_path, fname), lbl)

        assert expected == 2913, "unexpected dataset sizes"

### 1.2 Define the model architecture(2.0 point)
In this section you have the freedom to decide your own model. Keep in mind though, to perform image segmentation, you need to implement an architecture that does pixel level classification i.e. for each pixel in the image you need to predict the probability of it belonging to one of the 21 categories.

**We have implemented the U-Net architecture as introduced in the paper: https://arxiv.org/abs/1505.04597 with some additions**

In [4]:
## UNet Architecture With Valid convolutions and batch norm in the double convolution block.
# Valid conv. added to keep the output size same as the input image.
# Batch norm was introduced after the U-Net paper was published, so it'll only benefit in learning.

import torch.nn as nn

# Defining the block consisting of double convolution operations (each unpadded 3x3 convolution  followed by a relu activation)
# as used in U-Net paper.   
# in_c : number of input channels and out_c : number of output channels
def double_conv(in_c, out_c):
    conv = nn.Sequential(
    nn.Conv2d(in_c, out_c, kernel_size = 3, stride = 1, padding = 1, bias = False),
    nn.BatchNorm2d(out_c),
    nn.ReLU(inplace=True),
    nn.Conv2d(out_c, out_c, kernel_size = 3, stride = 1, padding = 1, bias = False),
    nn.BatchNorm2d(out_c),
    nn.ReLU(inplace=True),
    )
    return conv
    
# Cropping the feature map from the contracting path to concatentate it with corresponding
# feature map in the expansive path.
def crop_img(tensor, target_tensor):
    target_size = target_tensor.size()[2]
    tensor_size = tensor.size()[2]
    delta = tensor_size - target_size
    delta = delta // 2
    return tensor[:, :, delta:tensor_size-delta, delta:tensor_size-delta]
    
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        #Defining 2x2 max pooling layer with stride 2 for downsampling in contractive path
        self.max_pool_2x2 = nn.MaxPool2d(kernel_size = 2, stride = 2) 
        # 5 double convolution blocks with different number of filters/feature_channels
        self.down_conv_1 = double_conv(3,64)
        self.down_conv_2 = double_conv(64,128)
        self.down_conv_3 = double_conv(128,256)
        self.down_conv_4 = double_conv(256,512)
        self.down_conv_5 = double_conv(512,1024)
        
        # Up convolutions (2x2 up-convolutions) for the expansive path for upsamling
        self.up_trans_1 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2, stride=2)
        # Double convolutions in the expansive path
        self.up_conv_1 = double_conv(1024, 512)

        self.up_trans_2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2)        
        self.up_conv_2 = double_conv(512, 256)

        self.up_trans_3 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2, stride=2)        
        self.up_conv_3 = double_conv(256, 128)

        self.up_trans_4 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2)        
        self.up_conv_4 = double_conv(128, 64)
        
        # 1x1 convolution, out_channels = total desired classes (21 including bgnd for pascal voc)
        self.out = nn.Conv2d(in_channels=64, out_channels = 21, kernel_size = 1)

    def forward(self, image):
        # encoder
        # image--> batch_size, #imagechannels, height, width
        # First double conv block
        print(f'Original Image Size: {image.size()}')
        x1 = self.down_conv_1(image) # this map goes for concatenation
        print(f'Feature Map Size After 1st Double Conv: {x1.size()}')
        # First max pooling oper. taking the o/p of 1st double conv block 
        x2 = self.max_pool_2x2(x1) 
        x3 = self.down_conv_2(x2) # this map goes for concatenation
        x4 = self.max_pool_2x2(x3) 
        x5 = self.down_conv_3(x4) # this map goes for concatenation
        x6 = self.max_pool_2x2(x5) 
        x7 = self.down_conv_4(x6) # this map goes for concatenation
        x8 = self.max_pool_2x2(x7) 
        x9 = self.down_conv_5 (x8)
        print(f'Feature Map Size at the End of Contractive Path: {x9.size()}')
        
        # decoder
        x = self.up_trans_1(x9)
        # Cropping x7 to same size of x (feature map after up-sampling) for concatenation of both feature maps
        # Applying double convolution on the concatenated feature map
        x = self.up_conv_1(torch.cat([x,x7],1))
        
        x = self.up_trans_2(x)
        x = self.up_conv_2(torch.cat([x,x5],1))

        x = self.up_trans_3(x)
        x = self.up_conv_3(torch.cat([x,x3],1))
        
        x = self.up_trans_4(x)
        x = self.up_conv_4(torch.cat([x,x1],1))
        
        x = self.out(x)
        print(f'Output Image Size: {x.size()}')
        
        return x


In [5]:
# Just added this block for some testing (to check if the feature map dimensions are matching the U-Net architecture)
image = torch.rand((1,3,512,512))
model = UNet()
print(model(image))

Original Image Size: torch.Size([1, 3, 512, 512])
Feature Map Size After 1st Double Conv: torch.Size([1, 64, 512, 512])
Feature Map Size at the End of Contractive Path: torch.Size([1, 1024, 32, 32])
Output Image Size: torch.Size([1, 21, 512, 512])
tensor([[[[-0.3228,  0.2036,  0.0460,  ..., -0.0882,  0.4917,  0.4078],
          [-0.0702,  0.0019, -0.4617,  ...,  0.5635, -0.2774, -0.0257],
          [-0.0898,  0.3874,  0.4286,  ...,  0.4794, -0.2990,  0.4417],
          ...,
          [-0.5246,  0.3156, -0.6431,  ..., -0.0867, -0.0069,  0.3005],
          [-0.2187, -0.4853,  0.1621,  ...,  0.7068, -0.0453,  0.0082],
          [-0.1998, -0.2911,  0.0156,  ..., -0.1531,  0.0294,  0.0652]],

         [[ 0.0686,  0.1306, -0.0402,  ...,  0.4392,  0.4284,  0.4754],
          [-0.5010, -0.1275,  0.1959,  ..., -0.1242, -0.1144,  0.1549],
          [ 0.0267,  0.2691, -0.0065,  ...,  0.2441,  0.2245, -0.0407],
          ...,
          [-0.1258, -0.1820,  0.0326,  ..., -0.5058, -0.0435,  0.0772],


In [6]:
# Creating an instance of the model defined above. 
# You can modify it incase you need to pass paratemers to the constructor.
model = UNet()

In [7]:
# print(model)

In [8]:
"""# Using https://github.com/qubvel/segmentation_models.pytorch library for testing purposes
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=21,                      # model output channels (number of classes in your dataset)
)"""

'# Using https://github.com/qubvel/segmentation_models.pytorch library for testing purposes\nimport segmentation_models_pytorch as smp\n\nmodel = smp.Unet(\n    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7\n    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization\n    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)\n    classes=21,                      # model output channels (number of classes in your dataset)\n)'

### 1.3 Hyperparameters(0.5 points)
Define all the hyperparameters(not restricted to the three given below) that you find useful here.

In [9]:
local_path = './VOCtrainval_11-May-2012/VOCdevkit/VOC2012/' # modify it according to your device
bs = 1
epochs = 50 

### 1.4 Dataset and Dataloader(0.5 points)
Create the dataset using pascalVOCDataset class defined above. Use local_path defined in the cell above as root. 

In [10]:
# dataset variable
# augs = aug.Compose([aug.RandomRotate(10), aug.RandomHorizontallyFlip()])
# Can add these augmentations like here 
# [https://github.com/meetshah1995/pytorch-semseg/blob/801fb200547caa5b0d91b8dde56b837da029f746/ptsemseg/loader/pascal_voc_loader.py#L18]
dst = pascalVOCDataset(root=local_path, is_transform=True, split='trainval') # Can also create two loaders -- split b/w train and val

# dataloader variable
trainloader = data.DataLoader(dst, batch_size=bs)

print(f'Number of training images: {len(dst)}')

Number of training images: 2913


In [11]:
"""# Just added this cell for visualization

# i, d in enumerate(trainloader) ---> here, d is list
# img--d[0] has the shape [2,3,512,512] or [batch_sz, #img_channels, img_sz, img_sz] -- img batch
# label--d[1] has the shape [2, 512, 512] or [batch_sz, mask_sz, mask_sz] -- mask batch

for i, data in enumerate(trainloader):
    if i>10:
        break
    imgs, labels = data
    imgs = imgs.numpy()[:, ::-1, :, :]
    imgs = np.transpose(imgs, [0,2,3,1])
    f, axarr = plt.subplots(bs, 2)
    for j in range(bs):
        axarr[j][0].imshow(imgs[j])
        axarr[j][1].imshow(dst.decode_segmap(labels.numpy()[j]))
        plt.show()
    """

'# Just added this cell for visualization\n\n# i, d in enumerate(trainloader) ---> here, d is list\n# img--d[0] has the shape [2,3,512,512] or [batch_sz, #img_channels, img_sz, img_sz] -- img batch\n# label--d[1] has the shape [2, 512, 512] or [batch_sz, mask_sz, mask_sz] -- mask batch\n\nfor i, data in enumerate(trainloader):\n    if i>10:\n        break\n    imgs, labels = data\n    imgs = imgs.numpy()[:, ::-1, :, :]\n    imgs = np.transpose(imgs, [0,2,3,1])\n    f, axarr = plt.subplots(bs, 2)\n    for j in range(bs):\n        axarr[j][0].imshow(imgs[j])\n        axarr[j][1].imshow(dst.decode_segmap(labels.numpy()[j]))\n        plt.show()\n    '

### 1.5 Loss fuction and Optimizer(1.0 point)
Define below with the loss function you think would be most suitable for segmentation task. You are free to choose any optimizer to train the network.

In [12]:
# loss function
loss_f = nn.CrossEntropyLoss()

# optimizer variable
# opt = torch.optim.SGD(model.parameters(), weight_decay=5*1e-4, lr = 0.0001, momentum=0.9)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

### 1.6 Training the model(3.0 points)
Your task here is to complete the code below to perform a training loop and save the model weights after each epoch of training.

In [13]:
# Implementation from Abhishek Thakur book - pg 216
# Can also look at this training loop implementation:
# https://github.com/Dhr11/Semantic_Segmentation/blob/master/Main_src.py]

DEVICE = "cuda" #torch.device("cuda" if torch.cuda.is_available() else "cpu") # or simply "cuda"

model.to(DEVICE)

def train(dataset, data_loader, model, criterion, optimizer):
    """
    training function that trains for one epoch
    :param dataset: dataset class (SIIMDataset)
    :param data_loader: torch dataset loader
    :param model: model
    :param criterion: loss function
    :param optimizer: adam, sgd, etc.
    """
    # put the model in train mode
    model.train()
    # calculate number of batches
    num_batches = int(len(dataset) / data_loader.batch_size)
    # init tqdm to track progress
    tk0 = tqdm(data_loader, total=num_batches)
    # loop over all batches
    for d in tk0:
        # fetch input images and masks
        # from dataset batch
        inputs, targets = d
#         print(inputs.shape)
        # move images and masks to cpu/gpu device
        inputs = inputs.to(DEVICE, dtype=torch.float)
        targets = targets.to(DEVICE, dtype=torch.long)
        # zero grad the optimizer
        optimizer.zero_grad()
        # forward step of model
        outputs = model(inputs)
#         print(outputs.shape)
        # calculate loss
        loss = criterion(outputs, targets)
        # backward loss is calculated on a scaled loss
        # context since we are using mixed precision training
        # if you are not using mixed precision training,
        # you can use loss.backward() and delete the following
        # two lines of code
#         with amp.scale_loss(loss, optimizer) as scaled_loss:
#             scaled_loss.backward()
        loss.backward()
        # step the optimizer
        optimizer.step()
    # close tqdm
    tk0.close()

In [14]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for _ in range(epochs):
    for i, d in enumerate(trainloader):
    # your code goes here
        criterion = loss_f
        train(dst, trainloader, model, criterion, opt)


  0%|          | 1/2913 [00:00<09:35,  5.06it/s]

torch.Size([1, 3, 512, 512])
Original Image Size: torch.Size([1, 3, 512, 512])
Feature Map Size After 1st Double Conv: torch.Size([1, 64, 512, 512])
Feature Map Size at the End of Contractive Path: torch.Size([1, 1024, 32, 32])
Output Image Size: torch.Size([1, 21, 512, 512])
torch.Size([1, 21, 512, 512])
torch.Size([1, 3, 512, 512])


  0%|          | 2/2913 [00:00<11:48,  4.11it/s]

Original Image Size: torch.Size([1, 3, 512, 512])
Feature Map Size After 1st Double Conv: torch.Size([1, 64, 512, 512])
Feature Map Size at the End of Contractive Path: torch.Size([1, 1024, 32, 32])
Output Image Size: torch.Size([1, 21, 512, 512])
torch.Size([1, 21, 512, 512])
torch.Size([1, 3, 512, 512])


  0%|          | 3/2913 [00:00<16:05,  3.01it/s]

...

Original Image Size: torch.Size([1, 3, 512, 512])
Feature Map Size After 1st Double Conv: torch.Size([1, 64, 512, 512])
Feature Map Size at the End of Contractive Path: torch.Size([1, 1024, 32, 32])
Output Image Size: torch.Size([1, 21, 512, 512])
torch.Size([1, 21, 512, 512])
torch.Size([1, 3, 512, 512])


  1%|          | 29/2913 [00:11<20:21,  2.36it/s]

Original Image Size: torch.Size([1, 3, 512, 512])
Feature Map Size After 1st Double Conv: torch.Size([1, 64, 512, 512])
Feature Map Size at the End of Contractive Path: torch.Size([1, 1024, 32, 32])
Output Image Size: torch.Size([1, 21, 512, 512])
torch.Size([1, 21, 512, 512])
torch.Size([1, 3, 512, 512])


  1%|          | 29/2913 [00:12<20:25,  2.35it/s]


KeyboardInterrupt: 

### 1.7 Evaluate your model(1.5 points)
In this section you have to implement the evaluation metrics for your model. Calculate the values of F1-score, dice coefficient and AUC-ROC score on the data you used for training. You can use external packages like scikit-learn to compute above metrics.

In [ ]:
def evaluate(ground_truth, predictions):
    
    return f1_score, auc_score, dice_coeeficient

### 1.8 Plot the evaluation metrics against epochs(1.0)
In section 1.6 we saved the weights of the model after each epoch. In this section, you have to calculate the evaluation metrics after each epoch of training by loading the weights for each epoch. Once you have calculated the evaluation metrics for each epoch, plot them against the epochs.

### 1.9 Visualize results(0.5 points)
For any 10 images in the dataset, show the images along the with their segmentation mask.

**Helpful resources for part 2**

Cityscapes data loader similar to pascal voc: https://github.com/meetshah1995/pytorch-semseg/tree/801fb200547caa5b0d91b8dde56b837da029f746/ptsemseg/loader